In [70]:
import ee, geemap
from itertools import product

# Initialize Earth Engine
ee.Authenticate()
ee.Initialize()


In [398]:
# pip install folium shapely
import folium
from shapely.geometry import Polygon



okavango = [
    (21.7, -18.2),
    (21.7, -20.3),
    (23.6, -20.3),
    (23.6, -18.2),
]



poly = Polygon(okavango)
# Center map on polygon centroid (lat, lon order for Folium!)
lat, lon = poly.centroid.y, poly.centroid.x
m = folium.Map(location=[lat, lon], zoom_start=7, tiles="CartoDB positron")

# Folium wants [lat, lon] pairs; convert
coords_latlon = [(y, x) for x, y in okavango] + [(okavango[0][1], okavango[0][0])]
folium.Polygon(
    locations=coords_latlon,
    color="red",
    weight=3,
    fill=True,
    fill_opacity=0.2,
    tooltip="Okavango study area",
).add_to(m)

m


### Load Landsat DSWE ImageCollection

In [396]:

def list_image_ids(path: str):
    """
    return IDs of image assets under an Earth Engine path (filters out folders/collections)
    """
    assets = ee.data.listAssets(path).get('assets', [])
    return [a['id'] for a in assets if a.get('type') == 'IMAGE']

def list_ids(path: str):
    """
    return IDs of all assets under an Earth Engine path (filters out folders/collections)
    """
    assets = ee.data.listAssets(path).get('assets', [])
    return [a['id'] for a in assets]


# landsat monthly DSWE
ls_dswe_path = 'projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v2/DSWE_Products'
ls_dswe_ids  = list_image_ids(ls_dswe_path)
ls_dswe_ids[:10]


['projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v2/DSWE_Products/DSWE_1984_06',
 'projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v2/DSWE_Products/DSWE_1984_07',
 'projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v2/DSWE_Products/DSWE_1984_09',
 'projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v2/DSWE_Products/DSWE_1984_10',
 'projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v2/DSWE_Products/DSWE_1984_11',
 'projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v2/DSWE_Products/DSWE_1985_04',
 'projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v2/DSWE_Products/DSWE_1986_11',
 'projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v2/DSWE_Products/DSWE_1986_12',
 'projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v2/DSWE_Products/DSWE_1987_01',
 'projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v2/DSWE_Products/DSWE_1987_02']

In [324]:
def load_landsat_dswe(id):
    """
    Load Landsat DSWE image by asset ID, add a binary water mask, and tag date metadata.
    """
    year = ee.String(id).split('_').get(-2)
    month = ee.String(id).split('_').get(-1)
    img = ee.Image(id)
    binary = img.gt(0).toFloat().rename('dswe_bin')
    return img.addBands(binary).set({'year': year, 
            'month': month, 'date': ee.String(year).cat('-').cat(month)})

ls_dswe_collection = ee.ImageCollection([load_landsat_dswe(id) for id in ls_dswe_ids])
print (ls_dswe_collection.first().getInfo()['bands'])

[{'id': 'dswe', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [8320, 7112], 'crs': 'EPSG:4326', 'crs_transform': [0.00026949458523585647, 0, 21.791062667586118, 0, -0.00026949458523585647, -18.269307427723945]}, {'id': 'dswe_bin', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [8320, 7112], 'crs': 'EPSG:4326', 'crs_transform': [0.00026949458523585647, 0, 21.791062667586118, 0, -0.00026949458523585647, -18.269307427723945]}]


In [325]:
# Define palettes visualization parameters
palettes = {
    # sequential light→dark blues (ordered magnitudes; ColorBrewer "Blues"-like)
    "dswe_classes": ["white", "#deebf7", "#9ecae1", "#3182bd"],   # 1..4

    # binary white vs blue (crisp on dark/imagery basemaps)
    "dswe_binary":  ["white", "#3182bd"],                         # 0/1

    # zero-centered diverging brown↔teal with light neutral midpoint
    # (good for values around 0: residuals, anomalies, signed ratios)
    "centered":     ["#a6611a", "#dfc27d", "#f5f5f5", "#80cdc1", "#018571"],  # -1..1

    # classic diverging red↔white↔blue
    "diff_rwb":     ["#ef8a62", "#f7f7f7", "#67a9cf"]
}


vis_dswe_classes = {
    "palette": palettes["dswe_classes"],
    "min": 1,
    "max": 4
}

vis_dswe_binary = {
    "palette": palettes["dswe_binary"],
    "min": 0,
    "max": 1
}

vis_centered = {
    "palette": palettes["centered"],
    "min": -1,
    "max": 1
}

vis_diff_rwb = {
    "palette": palettes["diff_rwb"],
    "min": -1,
    "max": 1
}


In [326]:
#   Filters for year-specific and seasonal averages,
# Subset DSWE for comparison to Inman Lyons
#  TODO: check that DSWE and IL years match
dswe_compare = ls_dswe_collection  \
    .filter(ee.Filter.lte('month', '09')) \
    .filter(ee.Filter.gte('month', '07')) \
    .filter(ee.Filter.lt('year', '2025')) \

# Compute decade composites for DSWE
dswe_1990s = ls_dswe_collection \
    .filter(ee.Filter.lte('month', '09')) \
    .filter(ee.Filter.gte('month',  '07')) \
    .filter(ee.Filter.gte('year', '1985')) \
    .filter(ee.Filter.lte('year', '2005')) \
    .mean()

dswe_2010s = ls_dswe_collection \
    .filter(ee.Filter.gte('month', '07')) \
    .filter(ee.Filter.lte('month', '09')) \
    .filter(ee.Filter.gte('year', '2020')) \
    .filter(ee.Filter.lte('year', '2025')) \
    .mean()

dswe_change = dswe_2010s.subtract(dswe_1990s)


In [7]:
# --- export change image export 
run_exports = False

task_change = ee.batch.Export.image.toDrive(
    image=(
        dswe_change
        .select('dswe')                
        .unmask(0)
        .updateMask(dswe_1990s.select('dswe').eq(0))  
        .visualize(**vis_centered)     # use centered scheme for change
    ),
    description='DSWE_change_2010s_vs_1990s',
    folder='Okavango',
    fileNamePrefix='DSWE_change_2010s_vs_1990s',
    region=okavango,
    scale=30,
    crs='EPSG:4326',
    maxPixels=1e13
)

if run_exports:
    task_change.start()
else:
    print("Export created but not started")

Export created but not started


In [9]:

def export_landsat_dswe_monthly(collection,
                                  years=range(1985, 2025),
                                  months=range(1, 13),
                                  region=okavango,
                                  vis=vis_dswe_classes,
                                  folder='Okavango/monthly',
                                  start=False,
                                  skip_empty=False):
    """
    Export monthly Landsat DSWE composites:
      - Filters by 'year' (string) and 'month' ('01'..'12') properties
      - Averages matches (mean), visualizes with vis_dswe_classes
      - Writes TIFFs to Drive: landsat_DSWE_<year>_<month>

    Args:
      collection: ee.ImageCollection with 'dswe', 'year', 'month' properties
      years, months: ranges to iterate
      region: export geometry
      vis: visualization dict (default: vis_dswe_classes)
      folder: Drive folder
      start: if True, start tasks immediately
      skip_empty: if True, skip months with no images (extra .getInfo() per month)
    """
    for year in years:
        for month in months:
            month_str = f"{month:02d}"

            monthly = (collection
                       .filter(ee.Filter.eq('year', str(year)))
                       .filter(ee.Filter.eq('month', month_str)))

            if skip_empty and monthly.size().getInfo() == 0:
                print(f"Skip {year}-{month_str}: no scenes")
                continue

            comp = monthly.mean()
            image = (comp.select('dswe')
                          .unmask(0)
                          .visualize(**vis))

            desc = f'DSWE_{year}_{month_str}'
            fname = f'landsat_DSWE_{year}_{month_str}'

            task = ee.batch.Export.image.toDrive(
                image=image,
                description=desc,
                folder=folder,
                fileNamePrefix=fname,
                region=region,
                scale=30,
                crs='EPSG:4326',
                maxPixels=1e13
            )

            if start:
                task.start()
                print(f"Started export: {fname}")
            else:
                print(f"Created export (not started): {fname}")

# Example usage:
# export_landsat_dswe_monthly(ls_dswe_collection, start=run_exports, skip_empty=True)


## Load Sentinel DSWE ImageCollection

In [327]:
sent2_dswe_path = 'projects/ee-okavango/assets/water_masks/monthly_DSWE_Sent2_10m_v2/DSWE_Products'
sent2_dswe_ids  = list_image_ids(sent2_dswe_path)
sent2_dswe_ids[:10]

['projects/ee-okavango/assets/water_masks/monthly_DSWE_Sent2_10m_v2/DSWE_Products/DSWE_2018_12',
 'projects/ee-okavango/assets/water_masks/monthly_DSWE_Sent2_10m_v2/DSWE_Products/DSWE_2019_01',
 'projects/ee-okavango/assets/water_masks/monthly_DSWE_Sent2_10m_v2/DSWE_Products/DSWE_2019_02',
 'projects/ee-okavango/assets/water_masks/monthly_DSWE_Sent2_10m_v2/DSWE_Products/DSWE_2019_03',
 'projects/ee-okavango/assets/water_masks/monthly_DSWE_Sent2_10m_v2/DSWE_Products/DSWE_2019_04',
 'projects/ee-okavango/assets/water_masks/monthly_DSWE_Sent2_10m_v2/DSWE_Products/DSWE_2019_05',
 'projects/ee-okavango/assets/water_masks/monthly_DSWE_Sent2_10m_v2/DSWE_Products/DSWE_2019_06',
 'projects/ee-okavango/assets/water_masks/monthly_DSWE_Sent2_10m_v2/DSWE_Products/DSWE_2019_07',
 'projects/ee-okavango/assets/water_masks/monthly_DSWE_Sent2_10m_v2/DSWE_Products/DSWE_2019_08',
 'projects/ee-okavango/assets/water_masks/monthly_DSWE_Sent2_10m_v2/DSWE_Products/DSWE_2019_09']

In [328]:
def load_s2_dswe(id):
    """
    Load sentinel DSWE image by asset ID, add a binary water mask, and tag date metadata.
    """    
    year = ee.String(id).split('_').get(-2)
    month = ee.String(id).split('_').get(-1)
    img = ee.Image(id)
    binary = img.gt(0).toFloat().rename('DSWE_bin')
    return img.addBands(binary).set({'year': year, 'month': month, 'date': ee.String(year).cat('-').cat(month)})

s2_dswe = ee.ImageCollection([load_s2_dswe(id) for id in sent2_dswe_ids])
print (s2_dswe.first().getInfo()['properties']['year'])


# Filters to months used in IL 
s2_dswe_compare = s2_dswe \
    .filter(ee.Filter.lte('month', '09')) \
    .filter(ee.Filter.gte('month', '07')) \
    .filter(ee.Filter.lt('year', '2025')) \


# mean DSWE visualization export (10 m)
task_s2_mean = ee.batch.Export.image.toDrive(
    image = s2_dswe_compare.select('DSWE').mean().selfMask().visualize(**vis_dswe_classes),
    description    = 'DSWE_sentinel_mean_JulSep5',
    folder         = 'Okavango',
    fileNamePrefix = 'DSWE_sentinel_mean_JulSep',
    region         = okavango,
    scale          = 10,
    crs            = 'EPSG:4326',
    maxPixels      = 1e13
)



if run_exports:
    task_s2_mean.start()
    print("Started export: DSWE_sentinel_mean_JulSep")
else:
    print("Created export, not started: DSWE_sentinel_mean_JulSep")

2018
Created export, not started: DSWE_sentinel_mean_JulSep


In [329]:
def view_tasks(limit=10):
    """Print the first `limit` Earth Engine tasks with key metadata."""
    ops = ee.data.listOperations()[:limit]
    if not ops:
        print("No tasks found.")
    for i, op in enumerate(ops, 1):
        meta = op.get('metadata', {}) or {}
        state   = meta.get('state', '')
        desc    = meta.get('description', '(no desc)')
        create  = meta.get('createTime', '')
        update  = meta.get('updateTime', '')
        task_id = op.get('name', '')
        print(f"{i:2d}. {state:8}  {desc}")
        print(f"    id: {task_id}")
        print(f"    created: {create} | updated: {update}")


def cancel_tasks(limit=10, states=('READY','RUNNING','PENDING')):
    """Cancel the first `limit` tasks matching given states."""
    for op in ee.data.listOperations()[:limit]:
        meta = op.get('metadata', {}) or {}
        state = meta.get('state', '')
        name  = op.get('name', '')
        desc  = meta.get('description', '(no desc)')
        if state in states:
            ee.data.cancelOperation(name)
            print(f"Canceled: {desc} ({state})")
# see what’s queued
view_tasks(limit=10)

# cancel any READY/RUNNING tasks
# cancel_tasks(limit=5)

 1. SUCCEEDED  landsat_dswe_monthly_count
    id: projects/ee-okavango/operations/M6SB5BYCSKAQWA3L2UAF7MCX
    created: 2025-10-01T01:28:12.880113Z | updated: 2025-10-01T01:28:28.481171Z
 2. SUCCEEDED  landsat_dswe_monthly_count
    id: projects/ee-okavango/operations/CHGP5KFAIEJLJRWUNEHABKNF
    created: 2025-10-01T00:53:31.224972Z | updated: 2025-10-01T00:54:04.568349Z
 3. SUCCEEDED  okavango_LC_classification_09192025
    id: projects/ee-okavango/operations/7FV7WVR4AKGWEUUT7FH24D5J
    created: 2025-09-30T19:58:09.006113Z | updated: 2025-09-30T20:05:27.345129Z
 4. SUCCEEDED  dswe_2023_stack
    id: projects/ee-okavango/operations/KZIIVADBAYBLRAIG3AGCUKYC
    created: 2025-09-30T19:56:51.344340Z | updated: 2025-09-30T20:06:45.950632Z
 5. FAILED    dswe_2023_stack
    id: projects/ee-okavango/operations/CQI3TT6IQ2OQ6GE5U7MPRJXG
    created: 2025-09-30T19:52:00.543976Z | updated: 2025-09-30T19:52:14.098873Z
 6. SUCCEEDED  okavango_canopy_v3_10m
    id: projects/ee-okavango/operations/5

In [330]:
def explain_task(task_id=None):
    """
    Print details (state, description, error) for a given task_id,
    or for the most recent FAILED task if task_id is None.
    """
    ops = ee.data.listOperations()
    if not ops:
        print("No tasks found."); return

    if task_id:
        op = next((o for o in ops if o.get('name') == task_id), None)
        print (len(op))
        if not op:
            print(f"No task with id {task_id}"); 
            return
    else:
        failed = [o for o in ops if o.get('metadata', {}).get('state') == 'FAILED']
        if not failed:
            print("No failed tasks."); return
        op = max(failed, key=lambda o: o.get('metadata', {}).get('updateTime',''))
#         print (op['error']['message'], len(op))

    meta = op.get('metadata', {}) or {}
    err  = op.get('error', {})
    msg  = err.get('message') if isinstance(err, dict) else err

    print(f"desc   : {meta.get('description','(no desc)')}")
    print(f"id     : {op.get('name')}")
    print(f"state  : {meta.get('state')}")
    print(f"error  : {msg}")

explain_task()




desc   : IL_2024
id     : projects/ee-okavango/operations/A6PANZXSDEBSCMKYJBBJOTH5
state  : FAILED
error  : Image.clip: The area for image clipping must be a geometry, a Feature or a FeatureCollection.


In [ ]:

# Loop over years/months to export monthly Sentinel-2 DSWE composites
for year in range(2019, 2025):
    for month in range(1, 13):
        month_str = f"{month:02d}"

        # Filter the Sentinel-2 DSWE collection to this year+month and average all scenes
        monthly = (s2_dswe
            .filter(ee.Filter.eq('year', str(year)))
            .filter(ee.Filter.eq('month', month_str))
            .mean())

        print(monthly.getInfo())
        # visualization
        image = monthly.select('dswe').unmask(0).visualize(**vis_dswe_classes)

        # export task
        desc  = f'DSWE_{year}_{month_str}'
        fname = f's2_DSWE_{year}_{month_str}'

        # Filter the Sentinel-2 DSWE collection to this year+month and average all scenes        
        task = ee.batch.Export.image.toDrive(
            image=image,
            description=desc,
            folder='Okavango/monthly_sentinel',
            fileNamePrefix=fname,
            region=okavango,
            scale=10,
            crs='EPSG:4326',
            maxPixels=1e13
        )

        if run_exports:
            task.start()
            print(f"Started export: {fname}")
        else:
            print(f"Created export but did not start: {fname}")


{'type': 'Image', 'bands': [{'id': 'DSWE', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 255}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'DSWE_bin', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}
Started export: s2_DSWE_2019_01
{'type': 'Image', 'bands': [{'id': 'DSWE', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 255}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'DSWE_bin', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}
Started export: s2_DSWE_2019_02
{'type': 'Image', 'bands': [{'id': 'DSWE', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 255}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'DSWE_bin', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 

## Inman Lyons layers


 Inman and Lyons (2020) created their landcover/inundation maps using annual 
 Landsat composites from  July through September, the peak inundation period in
 the Okavango Delta.

 Composites are the median of all cloud/shadow-masked scenes in that window, 
 and gap-fill from ±1 year. Years with large gaps were dropped 
 (1993, 2000, 2009, 2010, 2012).
 For validation imagery, they inspected high-resolution images July–October 
 (broader than the composites) alongside the Landsat true-color composites.


In [332]:
def load_il_image(aid, band_name=None):
    """Load one Inman–Lyons image by asset id; optionally rename first band; tag 'year'."""
    img = ee.Image(aid)
    if band_name:
        img = img.select(0).rename(band_name)
    year = ee.String(aid).split('_').get(-1)
    return img.set({'year': year})

def load_il(path, band_name=None):
    """Load all IMAGE assets under a folder into an ImageCollection (client-side list)."""
    assets = ee.data.listAssets(path).get('assets', [])
    ids = [a['id'] for a in assets if a.get('type') == 'IMAGE']
    images = [load_il_image(i, band_name) for i in ids]  # build a Python list of ee.Image
    return ee.ImageCollection(images)


# --- IL collections ---
IL_collection = load_il('projects/ee-okavango/assets/Inman_Lyons/FloodArea')
IL_og         = load_il('projects/ee-okavango/assets/Inman_Lyons/Annual_inundation_maps',
                        band_name='SR_B7')
# prefer IL_og through 2018, then FloodArea
IL_merged     = IL_og.filter(ee.Filter.lt('year', '2019')).merge(IL_collection)  

# --- means & diffs (single-summary exports) ---
IL_mean      = IL_merged.mean()  # multi-year mean IL inundation
IL_mean_diff = IL_mean.subtract(dswe_compare.select('dswe_bin').mean()).rename('IL_minus_DSWE_mean')  # mean IL - mean DSWE


In [62]:
export_IL_mean = ee.batch.Export.image.toDrive(
    image = IL_mean.unmask(0).visualize(**vis_centered),   # centered palette for continuous values
    description='IL_mean',
    folder='Okavango',
    fileNamePrefix='IL_mean',
    region=okavango, scale=30, crs='EPSG:4326', maxPixels=1e13
)
export_IL_mean_diff = ee.batch.Export.image.toDrive(
    image=IL_mean_diff.unmask(0).visualize(**vis_diff_rwb),   # red–white–blue for signed differences
    description='IL_minus_DSWE_mean',
    folder='Okavango',
    fileNamePrefix='IL_minus_DSWE_mean',
    region=okavango, scale=30, crs='EPSG:4326', maxPixels=1e13
)


if run_exports:
    export_IL_mean.start()
    export_IL_mean_diff.start()
    print("Started exports: IL_mean, IL_minus_DSWE_mean")
else:
    print("Exports not started: IL_mean, IL_minus_DSWE_mean")


Started exports: IL_mean, IL_minus_DSWE_mean


In [63]:
# --- per-year exports ---
year0 = '1990'                                           # baseline year for diffs
base_1990 = IL_merged.filter(ee.Filter.eq('year', year0)).mean()

for y in range(1990, 2025):
    ys = str(y) # year string

    # yearly IL composite and matching DSWE (binary water) mean
    year_img  = IL_merged.filter(ee.Filter.eq('year', ys)).mean()
    dswe_year = dswe_compare.filter(ee.Filter.eq('year', ys)).mean()

    # export: IL yearly (visualize like DSWE binary for easy comparison)
    exp_year = ee.batch.Export.image.toDrive(
        image=year_img.unmask(0).clip(okavango).visualize(**vis_dswe_binary),
        description=f'IL_{ys}',
        folder='Okavango',
        fileNamePrefix=f'IL_{ys}',
        region=okavango, scale=30, crs='EPSG:4326', maxPixels=1e13
    )

    # export: (IL year) − (IL 1990) → change relative to baseline
    diff_1990 = year_img.subtract(base_1990).rename('IL_diff_1990')
    exp_diff = ee.batch.Export.image.toDrive(
        image=diff_1990.unmask(0).visualize(**vis_centered),
        description=f'IL_diff_1990_{ys}',
        folder='Okavango/IL_images',
        fileNamePrefix=f'IL_diff_1990_{ys}',
        region=okavango, scale=30, crs='EPSG:4326', maxPixels=1e13
    )

    # export: (IL year) − (DSWE year) → agreement/offset between IL and DSWE
    il_minus_dswe = year_img.subtract(dswe_year.select('dswe_bin')).rename('IL_minus_DSWE')
    exp_il_dswe = ee.batch.Export.image.toDrive(
        image=il_minus_dswe.unmask(0).visualize(**vis_diff_rwb),
        description=f'IL_minus_DSWE_{ys}',
        folder='Okavango',
        fileNamePrefix=f'IL_minus_DSWE_{ys}',
        region=okavango, scale=30, crs='EPSG:4326', maxPixels=1e13
    )

    if run_exports:
        exp_year.start(); 
        exp_diff.start(); 
        exp_il_dswe.start()
        print(f"Started: IL_{ys}, IL_diff_1990_{ys}, IL_minus_DSWE_{ys}")
    else:
        print(f"Created (not started): IL_{ys}, IL_diff_1990_{ys}, IL_minus_DSWE_{ys}")


Started: IL_1990, IL_diff_1990_1990, IL_minus_DSWE_1990
Started: IL_1991, IL_diff_1990_1991, IL_minus_DSWE_1991
Started: IL_1992, IL_diff_1990_1992, IL_minus_DSWE_1992
Started: IL_1993, IL_diff_1990_1993, IL_minus_DSWE_1993
Started: IL_1994, IL_diff_1990_1994, IL_minus_DSWE_1994
Started: IL_1995, IL_diff_1990_1995, IL_minus_DSWE_1995
Started: IL_1996, IL_diff_1990_1996, IL_minus_DSWE_1996
Started: IL_1997, IL_diff_1990_1997, IL_minus_DSWE_1997
Started: IL_1998, IL_diff_1990_1998, IL_minus_DSWE_1998
Started: IL_1999, IL_diff_1990_1999, IL_minus_DSWE_1999
Started: IL_2000, IL_diff_1990_2000, IL_minus_DSWE_2000
Started: IL_2001, IL_diff_1990_2001, IL_minus_DSWE_2001
Started: IL_2002, IL_diff_1990_2002, IL_minus_DSWE_2002
Started: IL_2003, IL_diff_1990_2003, IL_minus_DSWE_2003
Started: IL_2004, IL_diff_1990_2004, IL_minus_DSWE_2004
Started: IL_2005, IL_diff_1990_2005, IL_minus_DSWE_2005
Started: IL_2006, IL_diff_1990_2006, IL_minus_DSWE_2006
Started: IL_2007, IL_diff_1990_2007, IL_minus_DS

In [64]:

# toggles
run_exports = False          # start tasks now?
export_early_late = False    # also export early/late means?

year_ranges = [
    {'name': '1985_2000_vs_2005_2020', 'earlyStart': '1985', 'earlyEnd': '2000',
                                       'lateStart':  '2005', 'lateEnd':  '2020'},
    {'name': '1990_2005_vs_2010_2025', 'earlyStart': '1990', 'earlyEnd': '2005',
                                       'lateStart':  '2010', 'lateEnd':  '2025'},
    {'name': '1990_1995_vs_2020_2025', 'earlyStart': '1990', 'earlyEnd': '1995',
                                       'lateStart':  '2020', 'lateEnd':  '2025'},
    {'name': '2000_2005_vs_2020_2025', 'earlyStart': '2000', 'earlyEnd': '2005',  
                                       'lateStart':  '2020', 'lateEnd':  '2025'}
]

for r in year_ranges:
    early = (IL_merged
             .filter(ee.Filter.gte('year', r['earlyStart']))
             .filter(ee.Filter.lte('year', r['earlyEnd']))
             .mean())

    late  = (IL_merged
             .filter(ee.Filter.gte('year', r['lateStart']))
             .filter(ee.Filter.lte('year', r['lateEnd']))
             .mean())

    change = late.subtract(early).rename('IL_change')

    # change export (diverging/centered palette)
    exp_change = ee.batch.Export.image.toDrive(
        image=change.unmask(0).visualize(**vis_centered),
        description=f'IL_change_{r["name"]}',
        folder='Okavango',
        fileNamePrefix=f'IL_change_{r["name"]}',
        region=okavango, scale=30, crs='EPSG:4326', maxPixels=1e13
    )
    if run_exports:
        exp_change.start(); print(f"Started: IL_change_{r['name']}")
    else:
        print(f"Created (not started): IL_change_{r['name']}")

    if export_early_late:
        exp_early = ee.batch.Export.image.toDrive(
            image=early.unmask(0).visualize(**vis_dswe_classes),
            description=f'IL_{r["earlyStart"]}-{r["earlyEnd"]}',
            folder='Okavango',
            fileNamePrefix=f'IL_{r["earlyStart"]}-{r["earlyEnd"]}',
            region=okavango, scale=30, crs='EPSG:4326', maxPixels=1e13
        )
        exp_late = ee.batch.Export.image.toDrive(
            image=late.unmask(0).visualize(**vis_dswe_classes),
            description=f'IL_{r["lateStart"]}-{r["lateEnd"]}',
            folder='Okavango',
            fileNamePrefix=f'IL_{r["lateStart"]}-{r["lateEnd"]}',
            region=okavango, scale=30, crs='EPSG:4326', maxPixels=1e13
        )
        if run_exports:
            exp_early.start(); exp_late.start()
            print(f"Started: IL_{r['earlyStart']}-{r['earlyEnd']} and IL_{r['lateStart']}-{r['lateEnd']}")
        else:
            print(f"Created (not started): IL_{r['earlyStart']}-{r['earlyEnd']} and IL_{r['lateStart']}-{r['lateEnd']}")


Created (not started): IL_change_1985_2000_vs_2005_2020
Created (not started): IL_change_1990_2005_vs_2010_2025
Created (not started): IL_change_1990_1995_vs_2020_2025
Created (not started): IL_change_2000_2005_vs_2020_2025
